<a href="https://colab.research.google.com/github/khs5696/AI504/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Preprocessing

In [4]:
file_path = '/content/drive/MyDrive/연구/NLP/seq2seq/data/fra-eng/fra.txt'
lines = pd.read_csv(file_path, names=['src', 'tar', 'lic'], sep='\t')
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000]

# Addition of <sos>, <eos>
lines.tar = lines.tar.apply(lambda x : '\t' + x + '\n')

# Word_bag
src_set = set()
tar_set = set()
for src, tar in zip(lines.src, lines.tar):
  for char in src:
    src_set.add(char)
  for char in tar:
    tar_set.add(char)

src_vocab_size = len(src_set)+1
tar_vocab_size = len(tar_set)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

src_set = sorted(list(src_set))
tar_set = sorted(list(tar_set))

src_dict = dict([(src_set[i], i+1) for i in range(len(src_set))])
tar_dict = dict([(tar_set[i], i+1) for i in range(len(tar_set))])

encoded_src = []
for src in lines.src:
  elem = []
  for char in src:
    elem.append(src_dict[char])
  encoded_src.append(elem)

encoded_tar = []
for tar in lines.tar:
  elem = []
  for char in tar:
    elem.append(tar_dict[char])
  encoded_tar.append(elem)

decoded_tar = []
for elem in encoded_tar:
  decoded_tar.append(elem[1:])

max_src = max([len(src) for src in lines.src])
max_tar = max([len(tar) for tar in lines.tar])
print('source 문장의 최대 길이 :',max_src)
print('target 문장의 최대 길이 :',max_tar)

encoded_src = to_categorical(pad_sequences(encoded_src, maxlen=max_src, padding='post'))
encoded_tar = to_categorical(pad_sequences(encoded_tar, maxlen=max_tar, padding='post'))
decoded_tar = to_categorical(pad_sequences(decoded_tar, maxlen=max_tar, padding='post'))

source 문장의 char 집합 : 80
target 문장의 char 집합 : 104
source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 74


Model

In [5]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

encoder_inputs = Input(shape=(None, src_vocab_size))    # (None, None, 80)
encoder_lstm = LSTM(units=256, return_state=True)       # Hidden state size = 256

# Result of LSTM : hidden state & cell state
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)  # (None, 256)
encoder_states = [state_h, state_c]

# ******************************************
decoder_inputs = Input(shape=(None, tar_vocab_size))    # (None, None, 104)
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')

# encoder_states : Convext vector
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_outputs = decoder_softmax_layer(decoder_outputs)  # (None, None, 256) -> (None, None, 104)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

model.fit(x=[encoded_src, encoded_tar], y=decoded_tar, batch_size=64, epochs=40, validation_split=0.2)

Epoch 1/40
750/750 [==============================] - 22s 17ms/step - loss: 0.8631 - val_loss: 0.8020
Epoch 2/40
750/750 [==============================] - 10s 13ms/step - loss: 0.5892 - val_loss: 0.6809
Epoch 3/40
750/750 [==============================] - 10s 14ms/step - loss: 0.5154 - val_loss: 0.6179
Epoch 4/40
750/750 [==============================] - 10s 14ms/step - loss: 0.4656 - val_loss: 0.5638
Epoch 5/40
750/750 [==============================] - 11s 14ms/step - loss: 0.4277 - val_loss: 0.5262
Epoch 6/40
750/750 [==============================] - 10s 14ms/step - loss: 0.4008 - val_loss: 0.5062
Epoch 7/40
750/750 [==============================] - 10s 14ms/step - loss: 0.3804 - val_loss: 0.4820
Epoch 8/40
750/750 [==============================] - 10s 14ms/step - loss: 0.3640 - val_loss: 0.4632
Epoch 9/40
750/750 [==============================] - 10s 14ms/step - loss: 0.3504 - val_loss: 0.4517
Epoch 10/40
750/750 [==============================] - 11s 14ms/step - loss: 0.338

KeyboardInterrupt: ignored

In [5]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

index_to_src = dict((i, char) for char, i in src_dict.items())
index_to_tar = dict((i, char) for char, i in tar_dict.items())

In [6]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_dict['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoded_src[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 34ms/step
-----------------------------------
입력 문장: Go.
정답 문장: ouge !
번역 문장: ours !
1/1 [==============================] - 0s 29ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: onjour !
번역 문장: alut !
1/1 [==============================] - 0s 31ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: 'ai pigé !
번역 문장: ommence !
1/1 [==============================] - 0s 22ms/step
-----------------------------------
입력 문장: Go home.
정답 문장: entre à la maison.
번역 문장: llez !
1/1 [==============================] - 0s 19ms/step
-----------------------------------
입력 문장: Get going.
정답 문장: n avant.
번역 문장: écampe !
